In [1]:
import os
os.chdir('/Users/corentin/Documents/M2/BIPODS/Project/iPMCMC/ipmcmc')
%run generate_data.py
%run linear_gaussian_state_model
%run smc.py
%run csmc.py
%run ipmcmc.py

In [2]:

# 4.1. Linear Gaussian State Space Model
from scipy.spatial.transform import Rotation as R
np.random.seed(420)

# Parameters
r = R.from_rotvec(np.array([7*np.pi/10, 3*np.pi/10, np.pi/20]))
rotation_matrix = r.as_dcm()
scaling_matrix = 0.99*np.eye(3)
beta = np.random.dirichlet(np.ones(20)*0.2, 3).transpose()
alpha = scaling_matrix@rotation_matrix
t_max = 50
mu = np.array([0, 1, 1])
start_var = 0.1*np.eye(3)
omega = np.eye(3)
sigma = 0.1*np.eye(20)


n_particles = 100

linear_transition_model = [LinearMu(default_mean=mu, default_cov=start_var)]+[LinearTransition(default_mean=np.zeros(3),default_cov=omega,default_alpha=alpha) for t in range(1, t_max)]
linear_proposals = [LinearMu(default_mean=mu, default_cov=start_var)]+[LinearProposal(default_mean=np.zeros(3),default_cov=omega,default_alpha=alpha) for t in range(1, t_max)]
linear_observation_model = [LinearObservation(default_mean=np.zeros(20),default_cov=sigma,default_beta=beta) for t in range(0, t_max)]


# If we want to change the parameters
assert np.all(np.linalg.eigvals(start_var) > 0)
assert np.all(np.linalg.eigvals(omega) > 0)
assert np.all(np.linalg.eigvals(sigma) > 0)

l_states, l_observations = linear_gaussian_state_space(t_max=t_max, mu=mu, start_var=start_var, transition_var=omega, noise_var=sigma, transition_coeffs=alpha, observation_coeffs=beta)
display((l_states.shape, l_observations.shape))

((50, 3), (50, 20))

In [8]:
n_nodes = 32
n_conditional_nodes = 16
n_steps = 100
init_conditional_traj = np.zeros((n_conditional_nodes, t_max, len(mu)))

for i in range(n_conditional_nodes):
    particles, _, _ = smc(l_observations, n_particles,
       linear_transition_model, linear_proposals, linear_observation_model)
    init_conditional_traj[i] = particles.mean(axis=1)

In [9]:
particles, conditional_traj, weights, conditional_indices, zetas = ipmcmc(
    n_steps, n_nodes, n_conditional_nodes, l_observations, n_particles, init_conditional_traj,
    linear_proposals, linear_transition_model, linear_observation_model)

In [21]:
zetas.shape

(5, 16, 32)

In [10]:
from filterpy.kalman import KalmanFilter

fk = KalmanFilter(dim_x=3, dim_z=20)

fk.x = l_states[0]
fk.P = start_var

fk.F = alpha
fk.Q = omega

fk.H = beta
fk.R = sigma

mu_kal, cov_kal, _, _ = fk.batch_filter(l_observations)
means, covs, gain, predicted_covs = fk.rts_smoother(mu_kal, cov_kal)

In [11]:
w_barre = weights.mean(axis=2)

rao_black_traj = np.zeros((n_steps, n_conditional_nodes, t_max, len(mu)))
for r in range(n_steps):
    for p in range(n_conditional_nodes):
        for m in range(n_nodes):
            weighted_sum_parts = np.zeros((t_max, len(mu)))
            for n in range(n_particles):
                weighted_sum_parts += w_barre[r, m, n] * particles[r, m, :, n]
            weighted_zetas = zetas[r, p, m] * weighted_sum_parts
            rao_black_traj[r, p] += weighted_zetas

estimated_means_from_ipmcmc = np.mean(rao_black_traj, axis=(0,1))

In [12]:
estimated_means_from_ipmcmc

array([[-0.02221433,  0.59108266,  0.61764722],
       [ 0.59126659, -0.72495531, -0.47477185],
       [-0.12476874,  1.76530836,  0.12765403],
       [ 1.01683642, -0.24199294,  1.12270838],
       [ 0.71809757, -0.26632624, -1.87658915],
       [-0.86638581,  1.46217467,  1.29623443],
       [ 0.17859368, -1.74377801,  0.08835216],
       [ 0.22957764,  1.2719777 , -0.79675495],
       [ 1.33632513,  0.34191832,  1.59560516],
       [ 1.31958083,  0.29350167, -1.87688057],
       [-0.09809598,  2.45434947,  1.32258454],
       [ 2.22714532, -1.31569474,  0.7312841 ],
       [ 0.212001  ,  1.28101879, -1.9862668 ],
       [-0.65478273,  1.02616392,  2.78972669],
       [ 0.76121846, -2.91820959, -0.64082836],
       [-0.52347782,  1.75705073, -1.84382623],
       [-0.19295809, -0.18465212,  2.94717005],
       [ 0.80144457, -1.88071471, -2.02909623],
       [-0.60769888,  2.2670097 ,  0.5667877 ],
       [ 0.92431652, -0.84314879,  1.72801809],
       [ 1.40579111, -0.36372422, -1.868

In [13]:
means

array([[ 0.14858004,  0.57998754,  0.64302605],
       [ 0.92683102, -1.08946054, -0.76577551],
       [-0.15540975,  2.83209307,  0.18791207],
       [ 1.61174015, -0.39488628,  1.85869719],
       [ 1.10511093, -0.45705992, -3.10472598],
       [-1.39897983,  2.34652653,  2.09977192],
       [ 0.41590628, -2.76164292,  0.11561232],
       [ 0.77943559,  2.15790011, -1.16347841],
       [ 2.28642219,  0.52132178,  2.59081001],
       [ 2.14701786,  0.48756151, -3.14480296],
       [-0.16700461,  3.98072199,  2.13616737],
       [ 3.51969775, -2.16161285,  1.18469176],
       [ 0.21045678,  2.00059955, -3.35337127],
       [-1.26724023,  1.72627874,  4.56679496],
       [ 1.20696743, -4.714323  , -0.93921916],
       [-0.774977  ,  2.77686533, -3.04527654],
       [-0.3242391 , -0.26626125,  4.78328347],
       [ 1.31810637, -3.0217746 , -3.22845561],
       [-0.89263973,  3.61727654,  0.76351556],
       [ 1.45276874, -1.15388922,  2.96318993],
       [ 2.36513533, -0.64531128, -3.036